In [15]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder



In [23]:
import pandas as pd

# Load the data
train_data = pd.read_csv('Train-set.csv')
test_data = pd.read_csv('Test-set.csv')

# Convert 'yes' and 'no' in the target column to binary (1 and 0)
train_data['Target'] = train_data['Target'].map({'yes': 1, 'no': 0})

# Calculate correlations
correlations = train_data.corr()

# Print correlations with the target variable ("Target")
target_col_correlation = correlations['Target']
target_col_correlation = target_col_correlation.sort_values(ascending=False)

# Print the correlation values and info
print("Correlation of each feature with the target variable ('Target'):")
print(target_col_correlation)


ValueError: could not convert string to float: 'blue-collar'

In [24]:
import pandas as pd
import numpy as np

# Load the data
train_data = pd.read_csv('Train-set.csv')
test_data = pd.read_csv('Test-set.csv')

# Convert 'yes' and 'no' in the target column to binary (1 and 0)
train_data['Target'] = train_data['Target'].map({'yes': 1, 'no': 0})

# Select numeric columns for correlation calculation
numeric_columns = train_data.select_dtypes(include=['number']).columns

# Calculate correlations
correlations = train_data[numeric_columns].corr()

# Print correlations with the target variable ("Target")
target_col_correlation = correlations['Target']
target_col_correlation = target_col_correlation.sort_values(ascending=False)

# Print the correlation values and info
print("Correlation of each numeric feature with the target variable ('Target'):")
print(target_col_correlation)

# Handle categorical variables using one-hot encoding
categorical_columns = train_data.select_dtypes(include=['object']).columns
encoded_data = pd.get_dummies(train_data[categorical_columns], drop_first=True)

# Calculate correlations for one-hot encoded categorical variables
encoded_correlations = pd.concat([encoded_data, train_data['Target']], axis=1).corr()

# Print correlations with the target variable ("Target")
encoded_target_correlation = encoded_correlations['Target']
encoded_target_correlation = encoded_target_correlation.sort_values(ascending=False)

# Print the correlation values and info
print("\nCorrelation of each encoded categorical feature with the target variable ('Target'):")
print(encoded_target_correlation)


Correlation of each numeric feature with the target variable ('Target'):
Target        1.000000
duration      0.405814
previous      0.114603
Unnamed: 0    0.096272
balance       0.060058
age           0.027563
id           -0.002035
pdays        -0.047454
campaign     -0.069128
Name: Target, dtype: float64

Correlation of each encoded categorical feature with the target variable ('Target'):
Target              1.000000
poutcome_success    0.310858
month_oct           0.100405
month_mar           0.099643
month_sep           0.099401
                      ...   
month_may          -0.069294
poutcome_unknown   -0.071005
job_blue-collar    -0.072247
housing_yes        -0.075114
contact_unknown    -0.105630
Name: Target, Length: 93, dtype: float64


In [25]:
# Import necessary libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, OneHotEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from imblearn.over_sampling import BorderlineSMOTE
from sklearn.feature_selection import SelectFromModel

# Load the training and test data
train_data = pd.read_csv('Train-set.csv')
test_data = pd.read_csv('Test-set.csv')

# Separate the 'Target' column (labels) from the training data
y_train = train_data['Target']
train_data.drop('Target', axis=1, inplace=True)

# Combine train and test data for preprocessing
all_data = pd.concat([train_data, test_data], axis=0)

# Feature Engineering: Extract day of the week and create a weekend indicator
try:
    # Convert 'day' column to datetime
    all_data['day'] = pd.to_datetime(all_data['day'])
    # Extract day of the week (0-6) and create 'day_of_week' feature
    all_data['day_of_week'] = all_data['day'].dt.dayofweek
    # Create binary indicator for weekend (Saturday and Sunday)
    all_data['is_weekend'] = all_data['day_of_week'].isin([5, 6]).astype(int)
    # Drop the original 'day' column
    all_data.drop('day', axis=1, inplace=True)
except (ValueError, OverflowError, pd._libs.tslibs.np_datetime.OutOfBoundsDatetime):
    # Handle errors due to invalid date formats
    all_data['day'] = pd.to_datetime(all_data['day'], errors='coerce')
    all_data['day_of_week'] = all_data['day'].dt.dayofweek
    all_data['is_weekend'] = all_data['day_of_week'].isin([5, 6]).astype(int)
    all_data.drop('day', axis=1, inplace=True)

# Separate numeric and categorical columns
numeric_cols = all_data.select_dtypes(include=[np.number]).columns
categorical_cols = all_data.select_dtypes(include=[object]).columns

# Create transformers for numeric and categorical columns
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', RobustScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

# Preprocess the data using the column transformer
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numeric_cols),
    ('cat', categorical_transformer, categorical_cols)
])

X_all_preprocessed = preprocessor.fit_transform(all_data)

# Save the preprocessed and feature-engineered data to a CSV file
cleaned_data = pd.DataFrame(X_all_preprocessed, columns=list(numeric_cols) + list(preprocessor.named_transformers_['cat'].named_steps['encoder'].get_feature_names_out(categorical_cols)))
cleaned_data.to_csv('cleaned_data.csv', index=False)

# Handle class imbalance using BorderlineSMOTE
smote = BorderlineSMOTE(sampling_strategy='auto', random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_all_preprocessed[:train_data.shape[0]], y_train)

# Create and train optimized models
optimized_rf_model = RandomForestClassifier(n_estimators=150, max_depth=9, random_state=42)
optimized_rf_model.fit(X_train_resampled, y_train_resampled)


/var/folders/8p/b3ws2sh942n4mms732sp99fr0000gn/T/ipykernel_18235/296200383.py:27: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['day'] = pd.to_datetime(all_data['day'])
/var/folders/8p/b3ws2sh942n4mms732sp99fr0000gn/T/ipykernel_18235/296200383.py:36: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  all_data['day'] = pd.to_datetime(all_data['day'], errors='coerce')
/Users/macbookpro/Library/Python/3.9/lib/python/site-packages/sklearn/impute/_base.py:555: UserWarning: Skipping features without any observed values: ['day_of_week']. At least one non-missing value is needed for imputation with strategy='median'.
  warnings.warn(


ValueError: Shape of passed values is (78161, 1), indices imply (78161, 71)